In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [2]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [3]:
train =pd.read_csv('train.csv')
test =pd.read_csv('test.csv')
gabung = pd.concat([train, test], ignore_index=True)

In [4]:
gabung1=gabung[['text','target']]
df_balanced =gabung1

In [5]:
train = df_balanced.iloc[0:7613]
test = df_balanced.iloc[7613:10876]

In [16]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [18]:


# Fungsi preprocessing dan encoding tetap sama
def preprocess_and_encode(texts, batch_size=32):
    encoded_texts = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Preprocessing and Encoding"):
        batch_texts = texts[i:i+batch_size]
        preprocessed_text = bert_preprocess(batch_texts)
        encoded_text = bert_encoder(preprocessed_text)
        encoded_texts.append(encoded_text['pooled_output'].numpy())
    return np.concatenate(encoded_texts)

# Memproses teks dari dataset
texts = train['text'].tolist()
encoded_texts = preprocess_and_encode(texts)


Preprocessing and Encoding: 100%|████████████████████████████████████████████████████| 238/238 [16:02<00:00,  4.05s/it]


In [19]:
pooled_output = np.array(encoded_texts)

In [20]:
pooled_output.shape

(7613, 768)

In [24]:
y=train["target"]


0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
7608    1.0
7609    1.0
7610    1.0
7611    1.0
7612    1.0
Name: target, Length: 7613, dtype: float64

In [21]:
np.save('pooled_output_all.npy', pooled_output)

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(pooled_output, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=1/3, random_state=42)

# Membuat dataset TensorFlow untuk set pelatihan
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(10).prefetch(tf.data.AUTOTUNE)

# Membuat dataset TensorFlow untuk set validasi
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.batch(10).prefetch(tf.data.AUTOTUNE)


# Membuat dataset TensorFlow untuk set pengujian
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.batch(10).prefetch(tf.data.AUTOTUNE)

In [29]:
def create_model():
    
    input_layer = tf.keras.Input(shape=(768,), dtype=tf.float32, name='pooled_output')

    # Hidden layers
    dense1 = tf.keras.layers.Dense(128, activation='relu')(input_layer)
    dropout1 = tf.keras.layers.Dropout(0.3)(dense1)
    dense2 = tf.keras.layers.Dense(64, activation='relu')(dropout1)
    dropout2 = tf.keras.layers.Dropout(0.3)(dense2)
    
    # Output layer
    output = tf.keras.layers.Dense(1, activation='sigmoid')(dropout2)
    
    model = tf.keras.Model(inputs=input_layer, outputs=output)
    return model
# Membuat model
model = create_model()

# Kompilasi model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Latih model
#history = model.fit(dataset, epochs=35, verbose=1)
history = model.fit(X_train, y_train, 
                    validation_data=(X_val, y_val),
                    epochs=50, 
                    batch_size=10,
                    verbose=1)

Epoch 1/50
533/533 [==============================] - 1s 2ms/step - loss: 0.6845 - accuracy: 0.5577 - val_loss: 0.6260 - val_accuracy: 0.6735
Epoch 2/50
533/533 [==============================] - 1s 2ms/step - loss: 0.6396 - accuracy: 0.6271 - val_loss: 0.6116 - val_accuracy: 0.6715
Epoch 3/50
533/533 [==============================] - 1s 2ms/step - loss: 0.6252 - accuracy: 0.6523 - val_loss: 0.5930 - val_accuracy: 0.6932
Epoch 4/50
533/533 [==============================] - 1s 2ms/step - loss: 0.6170 - accuracy: 0.6709 - val_loss: 0.5862 - val_accuracy: 0.7070
Epoch 5/50
533/533 [==============================] - 1s 2ms/step - loss: 0.6038 - accuracy: 0.6748 - val_loss: 0.5814 - val_accuracy: 0.7109
Epoch 6/50
533/533 [==============================] - 1s 2ms/step - loss: 0.6044 - accuracy: 0.6825 - val_loss: 0.5762 - val_accuracy: 0.7240
Epoch 7/50
533/533 [==============================] - 1s 2ms/step - loss: 0.5960 - accuracy: 0.6921 - val_loss: 0.5730 - val_accuracy: 0.7240
Epoch 

In [34]:
# Menyimpan model
model.save('nama_model.h5')


In [30]:


loss, accuracy = model.evaluate(test_dataset, verbose=0)

# Print the test loss and accuracy
print(f'Test loss: {loss:.3f}')
print(f'Test accuracy: {accuracy:.3f}')

Test loss: 0.483
Test accuracy: 0.783


In [31]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [35]:

# Fungsi preprocessing dan encoding tetap sama
def preprocess_and_encode(texts, batch_size=32):
    encoded_texts = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Preprocessing and Encoding"):
        batch_texts = texts[i:i+batch_size]
        preprocessed_text = bert_preprocess(batch_texts)
        encoded_text = bert_encoder(preprocessed_text)
        encoded_texts.append(encoded_text['pooled_output'].numpy())
    return np.concatenate(encoded_texts)

# Memproses teks dari dataset
texts = test['text'].tolist()
encoded_texts = preprocess_and_encode(texts)


Preprocessing and Encoding: 100%|████████████████████████████████████████████████████| 102/102 [08:22<00:00,  4.93s/it]


In [36]:
pooled_output = np.array(encoded_texts)

In [41]:
pooled_output.shape

(3263, 768)

In [38]:
np.save('test.npy', pooled_output)

In [43]:
prediction_dataset = tf.data.Dataset.from_tensor_slices(pooled_output).batch(10)

In [44]:
predictions = model.predict(prediction_dataset)

327/327 [==============================] - 0s 672us/step


In [45]:
predictions 

array([[0.5458827 ],
       [0.43943316],
       [0.5966541 ],
       ...,
       [0.8780239 ],
       [0.4595592 ],
       [0.66802037]], dtype=float32)

In [46]:
# Konversi hasil prediksi probabilitas ke dalam prediksi biner (0 atau 1)
binary_predictions = (predictions > 0.5).astype(int)

# Menampilkan hasil prediksi biner
print("Prediksi biner (0 atau 1):", binary_predictions)


Prediksi biner (0 atau 1): [[1]
 [0]
 [1]
 ...
 [1]
 [0]
 [1]]


In [47]:
binary_predictions.shape


(3263, 1)

In [60]:


# Ubah array ke DataFrame
df = pd.DataFrame(binary_predictions, columns=['target'])

print(df)


      target
0          1
1          0
2          1
3          1
4          1
...      ...
3258       0
3259       1
3260       1
3261       0
3262       1

[3263 rows x 1 columns]


In [66]:
submission = pd.read_csv('submission.csv')
submission

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [67]:
submission = submission.drop(columns=['target'])
submission 

,id
0,0
1,2
2,3
3,9
4,11
...,...
3258,10861
3259,10865
3260,10868
3261,10874


In [68]:
submission=pd.concat([submission , df], axis=1)

In [69]:
submission

,id,target
0,0,1
1,2,0
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,0
3259,10865,1
3260,10868,1
3261,10874,0


In [73]:
submission.to_csv('submission1.csv', index=False)